In [5]:
import os
import tensorflow as tf
import numpy as np
import time
import datetime
import pickle
import random
import h5py
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Add, ReLU, LSTM, ConvLSTM2D
from tensorflow.keras.layers import Conv2D, Conv3D, MaxPooling2D, concatenate, Input, AveragePooling2D, TimeDistributed, Dropout
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.utils import register_keras_serializable
from config import *
# tf.config.run_functions_eagerly(True)
# TF_USE_LEGACY_KERAS=True

In [2]:
tf.__version__

'2.18.0'

In [3]:
# inputs
batch_size = 1 # this is total batchsize using all GPUs, so make divisible by num_gpus!
l_rate = 0.0001

# training data location
file_name_stub = 'hdf5_dm_test_auto_' # dm_july2021_ aim_july2021_expert_ dm_july2021_expert_
# file_name_stub = 'dm_6nov_aim_' 
folder_name = 'raw_data' 
starting_num = 1 # lowest file name to use in training
highest_num = 10 # highest file name to use in training 4000, 5500, 190, 45, 10

# whether to save model if training and where
model_name = 'ak47_m41a_55k_sub_drop_'
save_dir = 'save_model_BC'
SAVE_MODEL = True

# whether to resume training from a previous model
IS_LOAD_WEIGHTS_AND_MODEL=False
model_name = 'test_model_1'

In [4]:
input_shape[1:]

(150, 412, 3)

In [5]:
input_shape

(32, 150, 412, 3)

In [6]:
# from keras.layers import Reshape
# from keras.layers import GlobalAveragePooling1D
# start_time=time.time()

# if IS_LOAD_WEIGHTS_AND_MODEL:
#     # pick up training from earlier spot
#     print('-- loading model from saved file --')
#     model = tp_load_model(save_dir, weights_name)
#     # TODO: if .p file exists, load and check if matches current config
#     hypers_load = pickle.load(open(save_dir+'/'+weights_name+'.p', 'rb'))
#     print(hypers_load)

# else:
#     # useful tutorial for building, https://keras.io/getting-started/functional-api-guide/
#     print('-- building model from scratch --')

#     base_model = EfficientNetB0(weights='imagenet',input_shape=(input_shape[1:]),include_top=False)#input_shape[1:](150,412,3)
#     if 'randinit' in model_name:
#         print('random initialisation!\n\n') 
#         base_model = EfficientNetB0(weights='imagenet',input_shape=(input_shape[1:]),include_top=False)#input_shape[1:]
#     base_model.trainable = True

#     intermediate_model= Model(inputs=base_model.input, outputs=base_model.layers[161].output)
#     intermediate_model.trainable = True

#     # input_1 = Input(shape=(1,150,280,3),name='main_in')#input_shape ##
#     input_1 = Input(batch_shape=(None, 1,150,412,3),name='main_in')#input_shape ##
#     x = TimeDistributed(intermediate_model)(input_1) ##
#     x = ConvLSTM2D(filters=256,kernel_size=(3,3),stateful=False,return_sequences=True,dropout=0.5, recurrent_dropout=0.5)(x)
#     # x = TimeDistributed(Reshape((-1,)))(x)
#     # x = LSTM(256,stateful=False,return_sequences=True,dropout=0.2, recurrent_dropout=0.2)(x)
#     x = TimeDistributed(Flatten())(x) ##
#     # x = TimeDistributed(Dropout(0.5))(x)
#     # x = TimeDistributed(Dropout(0.5))(x)
#     # x = LSTM(256,stateful=False,return_sequences=True,dropout=0., recurrent_dropout=0.)(x)
#     # x = TimeDistributed(Dropout(0.5))(x)

#     dense_5 = x ##
#     output_1 = Dense(n_keys, activation='sigmoid')(dense_5) ##
#     output_2 = Dense(n_clicks, activation='sigmoid')(dense_5) ##
#     output_3 = Dense(n_mouse_x, activation='softmax')(dense_5) # softmax since mouse is mutually exclusive ##
#     output_4 = Dense(n_mouse_y, activation='softmax')(dense_5) ## 
#     output_5 = Dense(1, activation='linear')(dense_5)  ##
#     # output_all = concatenate([output_1,output_2,output_3,output_4], axis=-1)
#     output_all = concatenate([output_1,output_2,output_3,output_4,output_5], axis=-1) ##
#     # output_all = GlobalAveragePooling1D()(output_all)
#     model = Model(input_1, output_all)


In [7]:
strategy = tf.distribute.MirroredStrategy(["GPU:0"])
# strategy = tf.distribute.MirroredStrategy(["GPU:0","GPU:1","GPU:2", "GPU:3"])
print('\nnumber of devices using for training: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)

number of devices using for training: 1


I0000 00:00:1732392683.343899   75828 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [8]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


I0000 00:00:1732392683.534975   75828 gpu_device.cc:2022] Created device /device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


True

In [8]:
with strategy.scope():
    model_name = 'ak47_m41a_55k_sub_drop_'
    # useful tutorial for building, https://keras.io/getting-started/functional-api-guide/
    print('-- building model from scratch --')
    base_model = EfficientNetB0(weights='imagenet',input_shape=(input_shape[1:]),include_top=False)
    if 'randinit' in model_name:
        print('random initialisation!\n\n')
        base_model = EfficientNetB0(weights=None,input_shape=(input_shape[1:]),include_top=False)
    base_model.trainable = True

    intermediate_model= Model(inputs=base_model.input, outputs=base_model.layers[161].output)
    intermediate_model.trainable = True

    input_1 = Input(shape=input_shape,name='main_in')
    x = TimeDistributed(intermediate_model)(input_1)

    print('here1')
    if 'drop' in model_name:
        if 'big' in model_name:
            x = ConvLSTM2D(filters=512,kernel_size=(3,3),stateful=False,return_sequences=True,dropout=0.5, recurrent_dropout=0.5)(x)
        else:
            x = ConvLSTM2D(filters=256,kernel_size=(3,3),stateful=False,return_sequences=True,dropout=0.5, recurrent_dropout=0.5)(x)
    else:
        if 'big' in model_name:
            x = ConvLSTM2D(filters=512,kernel_size=(3,3),stateful=False,return_sequences=True)(x)
        else:
            x = ConvLSTM2D(filters=256,kernel_size=(3,3),stateful=False,return_sequences=True)(x)
    if 'extra' in model_name:
        if 'drop' in model_name:
            x = ConvLSTM2D(filters=256,kernel_size=(3,3),stateful=False,return_sequences=True,dropout=0.5, recurrent_dropout=0.5)(x)
        else:
            x = ConvLSTM2D(filters=256,kernel_size=(3,3),stateful=False,return_sequences=True)(x)
    print('here2')
    x = TimeDistributed(Flatten())(x)
    if 'LSTM' in model_name:
        if 'drop' in model_name:
            x = TimeDistributed(Dropout(0.5))(x)
            x = LSTM(256,stateful=False,return_sequences=True,dropout=0., recurrent_dropout=0.)(x)
            x = TimeDistributed(Dropout(0.5))(x)
        else:
            x = LSTM(256,stateful=False,return_sequences=True)(x)


    # 2) set up auxillary input,  which can have previous actions, as well as aux info like health, ammo, team
    # aux_input = Input(shape=(int(ACTIONS_PREV*(aux_input_length))),name='aux_in')
    # if AUX_INPUT_ON:
    #     flat = concatenate([flat, aux_input], axis=1)
    # else:
    #     pass

    # 3) add shared fc layers
    dense_5 = x

    # 4) set up outputs, sepearate outputs will allow seperate losses to be applied
    output_1 = TimeDistributed(Dense(n_keys, activation='sigmoid'))(dense_5)
    output_2 = TimeDistributed(Dense(n_clicks, activation='sigmoid'))(dense_5)
    output_3 = TimeDistributed(Dense(n_mouse_x, activation='softmax'))(dense_5) # softmax since mouse is mutually exclusive
    output_4 = TimeDistributed(Dense(n_mouse_y, activation='softmax'))(dense_5) 
    output_5 = TimeDistributed(Dense(1, activation='linear'))(dense_5) 
    # output_all = concatenate([output_1,output_2,output_3,output_4], axis=-1)
    output_all = concatenate([output_1,output_2,output_3,output_4,output_5], axis=-1)


    # 5) finish model definition
    if AUX_INPUT_ON:
        model = Model([input_1, aux_input], output_all)
    else:
        model = Model(input_1, output_all)

    print(model.summary())

    @register_keras_serializable(package="Custom", name="custom_loss")
    def custom_loss(y_true, y_pred):
        # y_true is shape [n_batch, n_timesteps, n_keys+n_clicks+n_mouse_x+n_mouse_y+n_reward+n_advantage]
        # where n_reward and n_advantage must =1
        # y_pred is shape [n_batch, n_timesteps, n_keys+n_clicks+n_mouse_x+n_mouse_y+n_val]
        # we'll use y_true to send in reward and eventually original advantage fn (or could recompute this?)
        # wasd keys
        loss1a = losses.binary_crossentropy(y_true[:,:,0:4], 
                                            y_pred[:,:,0:4])
        # space key
        loss1b = losses.binary_crossentropy(y_true[:,:,4:5], 
                                            y_pred[:,:,4:5])
        # reload key
        loss1c = losses.binary_crossentropy(y_true[:,:,n_keys-1:n_keys], 
                                            y_pred[:,:,n_keys-1:n_keys])

        # weapon switches, 1,2,3
        loss1d = losses.binary_crossentropy(y_true[:,:,n_keys-4:n_keys-1], 
                                            y_pred[:,:,n_keys-4:n_keys-1])

        # all other keys
        # loss1d = losses.binary_crossentropy(y_true[:,:,5:n_keys-1], 
        #                                     y_pred[:,:,5:n_keys-1])
        # left click
        loss2a = losses.binary_crossentropy(y_true[:,:,n_keys:n_keys+1], 
                                            y_pred[:,:,n_keys:n_keys+1])
        # right click
        loss2b = losses.binary_crossentropy(y_true[:,:,n_keys+1:n_keys+n_clicks], 
                                            y_pred[:,:,n_keys+1:n_keys+n_clicks])
        # mouse move x
        loss3 = losses.categorical_crossentropy(y_true[:,:,n_keys+n_clicks:n_keys+n_clicks+n_mouse_x], 
                                                y_pred[:,:,n_keys+n_clicks:n_keys+n_clicks+n_mouse_x])
        # mouse move y
        loss4 = losses.categorical_crossentropy(y_true[:,:,n_keys+n_clicks+n_mouse_x:n_keys+n_clicks+n_mouse_x+n_mouse_y], 
                                                y_pred[:,:,n_keys+n_clicks+n_mouse_x:n_keys+n_clicks+n_mouse_x+n_mouse_y])

        # critic loss -- measuring between consecutive time steps
        #  = ((reward_t + gamma  v_t+1) - v_t)^2
        # can't really decide whether to use reward_t or reward_t+1, but guess it doesn't matter too much
        loss_crit = 10*losses.MSE(y_true[:,:-1,n_keys+n_clicks+n_mouse_x+n_mouse_y:n_keys+n_clicks+n_mouse_x+n_mouse_y+1]
                            + GAMMA*y_pred[:,1:,n_keys+n_clicks+n_mouse_x+n_mouse_y:n_keys+n_clicks+n_mouse_x+n_mouse_y+1]
                            ,y_pred[:,:-1,n_keys+n_clicks+n_mouse_x+n_mouse_y:n_keys+n_clicks+n_mouse_x+n_mouse_y+1])

        return K.concatenate([loss1a, loss1b, loss1c, loss2a, loss3, loss4, loss_crit])
        # return K.concatenate([loss1a, loss2a, loss3, loss4])




    # metrics for each part of interest - useful for debugging
    @register_keras_serializable(package="Custom", name="wasd_acc")
    def wasd_acc(y_true, y_pred):
        return keras.metrics.binary_accuracy(y_true[:,:,0:4], y_pred[:,:,0:4])
    @register_keras_serializable(package="Custom", name="j_acc")
    def j_acc(y_true, y_pred): # other keys, space, ctrl, shift, 1,2,3, r
        return keras.metrics.binary_accuracy(y_true[:,:,4:5], y_pred[:,:,4:5])
    @register_keras_serializable(package="Custom", name="oth_keys_acc")
    def oth_keys_acc(y_true, y_pred): # other keys, space, ctrl, shift, 1,2,3, r
        return keras.metrics.binary_accuracy(y_true[:,:,5:n_keys], y_pred[:,:,5:n_keys])
    @register_keras_serializable(package="Custom", name="Lclk_acc")
    def Lclk_acc(y_true, y_pred):
        return keras.metrics.binary_accuracy(y_true[:,:,n_keys:n_keys+1], y_pred[:,:,n_keys:n_keys+1],threshold=0.5)
        # relative to proportion that don't fire 
        # return keras.metrics.binary_accuracy(y_true[:,n_keys:n_keys+1], y_pred[:,n_keys:n_keys+1],threshold=0.5) - (1 - keras.backend.mean(keras.backend.greater(y_true[:,n_keys:n_keys+1], 0.5)))
    @register_keras_serializable(package="Custom", name="Rclk_acc")
    def Rclk_acc(y_true, y_pred):
        return keras.metrics.binary_accuracy(y_true[:,:,n_keys+1:n_keys+n_clicks], y_pred[:,:,n_keys+1:n_keys+n_clicks],threshold=0.5)
    @register_keras_serializable(package="Custom", name="m_x_acc")
    def m_x_acc(y_true, y_pred):
        return keras.metrics.categorical_accuracy(y_true[:,:,n_keys+n_clicks:n_keys+n_clicks+n_mouse_x], 
                                                    y_pred[:,:,n_keys+n_clicks:n_keys+n_clicks+n_mouse_x])
    @register_keras_serializable(package="Custom", name="m_y_acc")
    def m_y_acc(y_true, y_pred):
        return keras.metrics.categorical_accuracy(y_true[:,:,n_keys+n_clicks+n_mouse_x:n_keys+n_clicks+n_mouse_x+n_mouse_y], 
                                                    y_pred[:,:,n_keys+n_clicks+n_mouse_x:n_keys+n_clicks+n_mouse_x+n_mouse_y])

    @register_keras_serializable(package="Custom", name="crit_mse")
    def crit_mse(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)  # Cast to float32
        y_pred = tf.cast(y_pred, tf.float32)  # Cast to float32
        return 100*losses.MSE(y_true[:,:-1,n_keys+n_clicks+n_mouse_x+n_mouse_y:n_keys+n_clicks+n_mouse_x+n_mouse_y+1]
                                                + GAMMA*y_pred[:,1:,n_keys+n_clicks+n_mouse_x+n_mouse_y:n_keys+n_clicks+n_mouse_x+n_mouse_y+1]
                                                ,y_pred[:,:-1,n_keys+n_clicks+n_mouse_x+n_mouse_y:n_keys+n_clicks+n_mouse_x+n_mouse_y+1])

    @register_keras_serializable(package="Custom", name="w_1")
    def w_1(y_true, y_pred):
        return keras.backend.mean(keras.backend.greater(y_true[:,:,0], 0.5))
    @register_keras_serializable(package="Custom", name="no_fire")
    def no_fire(y_true, y_pred):
        return 1 - keras.backend.mean(keras.backend.greater(y_true[:,:,n_keys:n_keys+1], 0.5))
    @register_keras_serializable(package="Custom", name="m_x_0")
    def m_x_0(y_true, y_pred):
        return keras.backend.mean(keras.backend.greater(y_true[:,:,n_keys+n_clicks+int(np.floor(n_mouse_x/2))], 0.5))
    @register_keras_serializable(package="Custom", name="m_y_0")
    def m_y_0(y_true, y_pred):
        return keras.backend.mean(keras.backend.greater(y_true[:,:,n_keys+n_clicks+n_mouse_x+int(np.floor(n_mouse_y/2))], 0.5))

    opt = optimizers.Adam(learning_rate=l_rate)
    # model.compile(loss=custom_loss,optimizer=opt, metrics=[Lclk_acc,no_fire,m_x_acc,m_x_0,m_y_acc,m_y_0])
    model.compile(loss=custom_loss,optimizer=opt, metrics=[Lclk_acc,no_fire,m_x_acc,m_y_acc,wasd_acc,crit_mse])
    print('successfully compiled model')

    

-- building model from scratch --
here1
here2


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ main_in             │ (None, 32, 150,   │          0 │ -                 │
│ (InputLayer)        │ 412, 3)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 32, 10,    │    867,975 │ main_in[0][0]     │
│ (TimeDistributed)   │ 26, 112)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_lstm2d         │ (None, 32, 8, 24, │  3,392,512 │ time_distributed… │
│ (ConvLSTM2D)        │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_1  │ (None, 32, 49152) │          0 │ conv_lstm2d[0][0] │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_2  │ (None, 32, 11)    │    540,683 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_3  │ (None, 32, 2)     │     98,306 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_4  │ (None, 32, 23)    │  1,130,519 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_5  │ (None, 32, 15)    │    737,295 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_6  │ (None, 32, 1)     │     49,153 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 32, 52)    │          0 │ time_distributed… │
│ (Concatenate)       │                   │            │ time_distributed… │
│                     │                   │            │ time_distributed… │
│                     │                   │            │ time_distributed… │
│                     │                   │            │ time_distributed… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,816,443 (26.00 MB)

 Trainable params: 6,800,276 (25.94 MB)

 Non-trainable params: 16,167 (63.16 KB)

None
successfully compiled model


In [9]:
N_JITTER = 20

In [10]:
class DataGenerator(keras.utils.Sequence):
# https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
    def __init__(self, list_IDs, batch_size=32, shuffle=True):
        self.list_IDs = list_IDs
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end() # run this once to start

    def __len__(self):
        # the number of batches per epoch - how many times are we calling this generator altogether
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        # generate one batch of data, index is the batch number

        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        X, y = self.__data_generation(list_IDs_temp)
        return (X, y)

    def on_epoch_end(self):
        # updates indexes after each epoch
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

        # could do subsampling at this stage, by 
        # using ID in format 'filenum-framenum-containkillevent'

    def __data_generation(self, list_IDs_temp):

        # set up empty arrays to fill
        x_shape = list(input_shape)
        x_shape.insert(0,self.batch_size)
        # y_shape = [self.batch_size,n_keys+n_clicks+n_mouse_x+n_mouse_y]
        y_shape = [self.batch_size,N_TIMESTEPS,n_keys+n_clicks+n_mouse_x+n_mouse_y+1+1] # add 1 for reward, 1 for adv

        X = np.empty(x_shape)
        print(x.shape)
        y = np.empty(y_shape)

        for i, ID in enumerate(list_IDs_temp): 
            # print(i,end='\r')
            # ID is in format 'filenum-framenum'
            ID = ID.split('-')
            file_num = int(ID[0])
            frame_num = int(ID[1])+np.random.randint(0,N_JITTER-1)
            frame_num = np.minimum(frame_num,999-N_TIMESTEPS)
            frame_num = np.maximum(frame_num,0)

            # quicker way reading from hdf5
            file_name = folder_name +'/'+file_name_stub + str(file_num) + '.hdf5'
            h5file = h5py.File(file_name, 'r')

            for j in range(N_TIMESTEPS):
                X[i,j] = h5file['frame_'+str(frame_num+j)+'_x'][:] # /255
                y[i,j,:-2] = h5file['frame_'+str(frame_num+j)+'_y'][:]

                help_i = h5file['frame_'+str(frame_num+j)+'_helperarr'][:]
                kill_i = help_i[0]
                dead_i = help_i[1]
                shoot_i = y[i,j,n_keys:n_keys+1] # all these are binary variables
                reward_i = kill_i - 0.5*dead_i - 0.01*shoot_i # this is reward function
                y[i,j,-2:] = ((reward_i[0]),0.) # 0. is a placeholder for original advantage

                # for mouse, we're going to use a manual hack to remove most extreme 2 classes
                if y[i,j,n_keys+n_clicks] == 1:
                    y[i,j,n_keys+n_clicks] = 0
                    y[i,j,n_keys+n_clicks+2] = 1
                elif y[i,j,n_keys+n_clicks+1] == 1:
                    y[i,j,n_keys+n_clicks+1] = 0
                    y[i,j,n_keys+n_clicks+2] = 1
                elif y[i,j,n_keys+n_clicks+n_mouse_x-1] == 1:
                    y[i,j,n_keys+n_clicks+n_mouse_x-1] = 0
                    y[i,j,n_keys+n_clicks+n_mouse_x-3] = 1
                elif y[i,j,n_keys+n_clicks+n_mouse_x-2] == 1:
                    y[i,j,n_keys+n_clicks+n_mouse_x-2] = 0
                    y[i,j,n_keys+n_clicks+n_mouse_x-3] = 1

                # same for mouse y as of 20 aug
                if y[i,j,n_keys+n_clicks+n_mouse_x] == 1:
                    y[i,j,n_keys+n_clicks+n_mouse_x] = 0
                    y[i,j,n_keys+n_clicks+n_mouse_x+2] = 1
                elif y[i,j,n_keys+n_clicks+n_mouse_x+1] == 1:
                    y[i,j,n_keys+n_clicks+n_mouse_x+1] = 0
                    y[i,j,n_keys+n_clicks+n_mouse_x+2] = 1
                elif y[i,j,n_keys+n_clicks+n_mouse_x+n_mouse_y-1] == 1:
                    y[i,j,n_keys+n_clicks+n_mouse_x+n_mouse_y-1] = 0
                    y[i,j,n_keys+n_clicks+n_mouse_x+n_mouse_y-3] = 1
                elif y[i,j,n_keys+n_clicks+n_mouse_x+n_mouse_y-2] == 1:
                    y[i,j,n_keys+n_clicks+n_mouse_x+n_mouse_y-2] = 0
                    y[i,j,n_keys+n_clicks+n_mouse_x+n_mouse_y-3] = 1

            # add a manual hack here to make sure lclick is down  2 aug 2021
            # this is because firing rate of guns is slower than frame rate
            # yes I know should have done this at preprocessing stage...
            for j in range(1,N_TIMESTEPS-1):
                if y[i,j-1,n_keys:n_keys+1] == 1 and y[i,j+1,n_keys:n_keys+1] == 1:
                    y[i,j,n_keys:n_keys+1] = 1

            # 7 aug seem to need to fill in 1001 as well for spraying
            for j in range(1,N_TIMESTEPS-2):
                if y[i,j-1,n_keys:n_keys+1] == 1 and y[i,j+2,n_keys:n_keys+1] == 1:
                    y[i,j,n_keys:n_keys+1] = 1
                    y[i,j+1,n_keys:n_keys+1] = 1

            # TODO, include x_aux

            h5file.close()

            # do data aug
            # have the choice of mirroring image 
            # and accompanying mouse movement
            # this seemed to work ok for aim mode, but not deathmatch
            if IS_MIRROR:
                if np.random.rand()<0.3:
                    X[i] = np.flip(X[i],-2) # flip width dim
                    # also need to flip mouse x movement
                    y[i,:,n_keys+n_clicks:n_keys+n_clicks+n_mouse_x] = np.flip(y[i,:,n_keys+n_clicks:n_keys+n_clicks+n_mouse_x],axis=-1)
                    # also must flip 'a' and 'd' keys
                    akey = y[i,:,1]
                    dkey = y[i,:,3]
                    y[i,:,1] = dkey
                    y[i,:,3] = akey
            if True:
                # brightness
                if np.random.rand()<0.5: # was 0.2, raised to 0.5
                    # adjust in range 0.7 to 1.1, <1 darkesns, >1 brightens
                    bright = np.random.rand()*0.6+0.7
                    X[i] *= bright
                    X[i] = np.clip(X[i],0,255).astype(int)

                # contrast
                # follow https://stackoverflow.com/questions/49142561/change-contrast-in-numpy/49142934
                if np.random.rand()<0.5:
                    contrast = np.random.rand()*0.6+0.7
                    X[i] = np.clip(128 + contrast * X[i] - contrast * 128, 0, 255).astype(int)

        return (X, y)

In [11]:
data_list1 = [str(x1)+'-'+str(x2) for x1 in np.arange(starting_num,highest_num+1) for x2 in np.arange(0,1000-N_TIMESTEPS-int(N_JITTER),N_TIMESTEPS)]
print(data_list1)
data_list2 = [str(x1)+'-'+str(x2) for x1 in np.arange(starting_num,highest_num+1) for x2 in np.arange(0,1000-N_TIMESTEPS-int(N_JITTER),N_TIMESTEPS)]
data_list3 = [str(x1)+'-'+str(x2) for x1 in np.arange(starting_num,highest_num+1) for x2 in np.arange(0,1000-N_TIMESTEPS-int(N_JITTER),N_TIMESTEPS)]
data_list4 = [str(x1)+'-'+str(x2) for x1 in np.arange(starting_num,highest_num+1) for x2 in np.arange(0,1000-N_TIMESTEPS-int(N_JITTER),N_TIMESTEPS)]
data_list_full = [str(x1)+'-'+str(x2) for x1 in np.arange(starting_num,highest_num+1) for x2 in np.arange(0,1000-N_TIMESTEPS-int(N_JITTER),N_TIMESTEPS)]

['1-0', '1-32', '1-64', '1-96', '1-128', '1-160', '1-192', '1-224', '1-256', '1-288', '1-320', '1-352', '1-384', '1-416', '1-448', '1-480', '1-512', '1-544', '1-576', '1-608', '1-640', '1-672', '1-704', '1-736', '1-768', '1-800', '1-832', '1-864', '1-896', '1-928', '2-0', '2-32', '2-64', '2-96', '2-128', '2-160', '2-192', '2-224', '2-256', '2-288', '2-320', '2-352', '2-384', '2-416', '2-448', '2-480', '2-512', '2-544', '2-576', '2-608', '2-640', '2-672', '2-704', '2-736', '2-768', '2-800', '2-832', '2-864', '2-896', '2-928', '3-0', '3-32', '3-64', '3-96', '3-128', '3-160', '3-192', '3-224', '3-256', '3-288', '3-320', '3-352', '3-384', '3-416', '3-448', '3-480', '3-512', '3-544', '3-576', '3-608', '3-640', '3-672', '3-704', '3-736', '3-768', '3-800', '3-832', '3-864', '3-896', '3-928', '4-0', '4-32', '4-64', '4-96', '4-128', '4-160', '4-192', '4-224', '4-256', '4-288', '4-320', '4-352', '4-384', '4-416', '4-448', '4-480', '4-512', '4-544', '4-576', '4-608', '4-640', '4-672', '4-704', '4

In [12]:

print('data_list1 training on sequences: ',len(data_list1))
print('data_list1 training on frames: ',len(data_list1*N_TIMESTEPS))

print('data_list2 training on sequences: ',len(data_list2))
print('data_list2 training on frames: ',len(data_list2*N_TIMESTEPS))

print('data_list3 training on sequences: ',len(data_list3))
print('data_list3 training on frames: ',len(data_list3*N_TIMESTEPS))

print('data_list4 training on sequences: ',len(data_list4))
print('data_list4 training on frames: ',len(data_list4*N_TIMESTEPS))


np.random.shuffle(data_list1) # shuffle it (in place)
partition1 = {}
partition1['train'] = data_list1[:int(len(data_list1)*1.)]
partition1['validation'] = data_list1[int(len(data_list1)*0.995):]

np.random.shuffle(data_list2) # shuffle it (in place)
partition2 = {}
partition2['train'] = data_list2[:int(len(data_list2)*1.)]
partition2['validation'] = data_list2[int(len(data_list2)*0.995):]

np.random.shuffle(data_list3) # shuffle it (in place)
partition3 = {}
partition3['train'] = data_list3[:int(len(data_list3)*1.)]
partition3['validation'] = data_list3[int(len(data_list3)*0.995):]

np.random.shuffle(data_list4) # shuffle it (in place)
partition4 = {}
partition4['train'] = data_list4[:int(len(data_list4)*1.)]
partition4['validation'] = data_list4[int(len(data_list4)*0.995):]

# this is not subsampled
partition_full = {}
partition_full['tmp'] = data_list_full[:int(batch_size*2)]
partition_full['train_full'] = data_list_full[:int(len(data_list_full)*1.)]
partition_full['validation_full'] = data_list_full[int(len(data_list_full)*0.995):]


data_list1 training on sequences:  300
data_list1 training on frames:  9600
data_list2 training on sequences:  300
data_list2 training on frames:  9600
data_list3 training on sequences:  300
data_list3 training on frames:  9600
data_list4 training on sequences:  300
data_list4 training on frames:  9600


In [13]:
# training_generator1 = data_generation_self_made(partition1['train'], batch_size=batch_size, shuffle=True)
# validation_generator1 = data_generation_self_made(partition1['validation'], batch_size=batch_size, shuffle=True)

In [14]:
training_generator1 = DataGenerator(list_IDs=partition1['train'], batch_size=batch_size, shuffle=True)
validation_generator1 = DataGenerator(list_IDs=partition1['validation'], batch_size=batch_size, shuffle=True)

training_generator2 = DataGenerator(list_IDs=partition2['train'], batch_size=batch_size, shuffle=True)
validation_generator2 = DataGenerator(list_IDs=partition2['validation'], batch_size=batch_size, shuffle=True)

training_generator3 = DataGenerator(list_IDs=partition3['train'], batch_size=batch_size, shuffle=True)
validation_generator3 = DataGenerator(list_IDs=partition3['validation'], batch_size=batch_size, shuffle=True)

training_generator4 = DataGenerator(list_IDs=partition4['train'], batch_size=batch_size, shuffle=True)
validation_generator4 = DataGenerator(list_IDs=partition4['validation'], batch_size=batch_size, shuffle=True)

tmp_generator = DataGenerator(list_IDs=partition_full['tmp'], batch_size=batch_size, shuffle=True)
training_generator_full = DataGenerator(list_IDs=partition_full['train_full'], batch_size=batch_size, shuffle=True)
validation_generator_full = DataGenerator(list_IDs=partition_full['validation_full'], batch_size=batch_size, shuffle=True)

In [15]:
training_generator1[0]

NameError: name 'x' is not defined

In [16]:
# hist = model.fit(training_generator1,epochs=1,verbose=1)

In [17]:
# hist = model.fit(training_generator1,epochs=1,verbose=1)

In [2]:
import os
import pickle
from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import Optimizer

def tp_save_model(model, save_dir, model_name):
    # Create directory if it doesn't exist
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)

    # Save model architecture to JSON
    model_json = model.to_json()
    model_path = os.path.join(save_dir, model_name + '.json')
    with open(model_path, 'w') as json_file:
        json_file.write(model_json)
    print('Saved model architecture to', model_path)

    # Save weights to HDF5 with updated naming convention
    weights_path = os.path.join(save_dir, model_name + '.weights.h5')
    model.save_weights(weights_path)
    print('Saved weights to', weights_path)

    # Save optimizer state
    optimizer_weights = getattr(model.optimizer, 'weights', [])
    optimizer_path = os.path.join(save_dir, model_name + '_opt.pkl')
    with open(optimizer_path, 'wb') as opt_file:
        pickle.dump(optimizer_weights, opt_file)
    print('Saved optimizer state to', optimizer_path)

    # Save hyperparameters
    hypers = {
        'csgo_img_dimension': csgo_img_dimension,
        'csgo_game_res': csgo_game_res,
        'FRAMES_STACK': FRAMES_STACK,
        'N_TIMESTEPS': N_TIMESTEPS,
        'FRAMES_SKIP': FRAMES_SKIP,
        'ACTIONS_PREV': ACTIONS_PREV,
        'AUX_INPUT_ON': AUX_INPUT_ON,
        'mouse_x_possibles': mouse_x_possibles,
        'mouse_y_possibles': mouse_y_possibles,
        'n_keys': n_keys,
        'n_clicks': n_clicks,
        'n_extras': n_extras,
        'aux_input_length': aux_input_length,
    }
    hypers_path = os.path.join(save_dir, model_name + '_hypers.pkl')
    with open(hypers_path, 'wb') as hypers_file:
        pickle.dump(hypers, hypers_file)
    print('Saved hyperparameters to', hypers_path)

    return


def tp_load_model(save_dir, model_name):

    # Load model architecture from JSON
    model_path = os.path.join(save_dir, model_name + '.json')
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model JSON file not found: {model_path}")
    with open(model_path, 'r') as json_file:
        model_json = json_file.read()

    # Add all custom functions and metrics here
    custom_objects = {
        "custom_loss": custom_loss,
        "wasd_acc": wasd_acc,
        "j_acc": j_acc,
        "oth_keys_acc": oth_keys_acc,
        "Lclk_acc": Lclk_acc,
        "Rclk_acc": Rclk_acc,
        "m_x_acc": m_x_acc,
        "m_y_acc": m_y_acc,
        "crit_mse": crit_mse,
        "w_1": w_1,
        "no_fire": no_fire,
        "m_x_0": m_x_0,
        "m_y_0": m_y_0,
    }

    # Load the model with custom objects
    model = model_from_json(model_json, custom_objects=custom_objects)
    print('Loaded model architecture from', model_path)

    # Load weights from HDF5
    weights_path = os.path.join(save_dir, model_name + '.weights.h5')
    if not os.path.exists(weights_path):
        raise FileNotFoundError(f"Model weights file not found: {weights_path}")
    model.load_weights(weights_path)
    print('Loaded weights from', weights_path)

    return model


In [19]:
len(training_generator1), len(validation_generator1)

(300, 2)

In [20]:
# hist = model.fit(training_generator1,epochs=1,verbose=1)
# tp_save_model(model, save_dir, 'model_10file_timestep_16_1')

/home/tuan/.pyenv/versions/3.12.0/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


(None, 32, 49152)
(None, 32, 49152)
(None, 32, 49152)
(None, 32, 49152)
(None, 32, 49152)
(None, 32, 49152)
(None, 32, 49152)


E0000 00:00:1732386067.812690   54641 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1_1/time_distributed_1/block2b_drop_63/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


(None, 32, 49152)
(None, 32, 49152)


I0000 00:00:1732386122.133541   54736 cuda_dnn.cc:529] Loaded cuDNN version 90300


(None, 32, 49152)
(None, 32, 49152)
(None, 32, 49152)
(None, 32, 49152)
  1/300 ━━━━━━━━━━━━━━━━━━━━ 12:18:22 148s/step - crit_mse: 4.4160 - lclk_acc: 0.4375 - loss: 1.3988 - m_x_acc: 0.0000e+00 - m_y_acc: 0.0312 - no_fire: 0.6875 - wasd_acc: 0.4062(None, 32, 49152)
  2/300 ━━━━━━━━━━━━━━━━━━━━ 7:27 2s/step - crit_mse: 7.1776 - lclk_acc: 0.4375 - loss: 1.4118 - m_x_acc: 0.0000e+00 - m_y_acc: 0.0703 - no_fire: 0.5156 - wasd_acc: 0.4453      (None, 32, 49152)
  3/300 ━━━━━━━━━━━━━━━━━━━━ 7:37 2s/step - crit_mse: 8.2955 - lclk_acc: 0.4722 - loss: 1.4092 - m_x_acc: 0.0000e+00 - m_y_acc: 0.0747 - no_fire: 0.4201 - wasd_acc: 0.4696(None, 32, 49152)
  4/300 ━━━━━━━━━━━━━━━━━━━━ 7:26 2s/step - crit_mse: 8.8137 - lclk_acc: 0.5007 - loss: 1.4004 - m_x_acc: 0.0039 - m_y_acc: 0.0794 - no_fire: 0.3581 - wasd_acc: 0.4909    (None, 32, 49152)
  5/300 ━━━━━━━━━━━━━━━━━━━━ 7:17 1s/step - crit_mse: 9.3063 - lclk_acc: 0.5168 - loss: 1.3948 - m_x_acc: 0.0106 - m_y_acc: 0.0835 - no_fire: 0.3140 - wasd_acc:

2024-11-24 01:29:20.439256: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-24 01:29:20.439347: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


300/300 ━━━━━━━━━━━━━━━━━━━━ 584s 1s/step - crit_mse: 10.3118 - lclk_acc: 0.8928 - loss: 0.9863 - m_x_acc: 0.1870 - m_y_acc: 0.4937 - no_fire: 0.0809 - wasd_acc: 0.7617
Saved model architecture to save_model_BC/model_10file_timestep_16_1.json
Saved weights to save_model_BC/model_10file_timestep_16_1.weights.h5
Saved optimizer state to save_model_BC/model_10file_timestep_16_1_opt.pkl
Saved hyperparameters to save_model_BC/model_10file_timestep_16_1_hypers.pkl


In [25]:
# # hist = model.fit(training_generator1,epochs=1,workers=4,verbose=1,use_multiprocessing=True, max_queue_size=20)
# iter_letter = 'a'
# # hist = model.fit(training_generator1[0], training_generator1[1], epochs=1,verbose=1)

# hist = model.fit(training_generator1, validation_data=validation_generator1,epochs=1,verbose=1) 

In [26]:
#hist = model.fit(training_generator_full,validation_data=validation_generator_full,epochs=1,workers=4,verbose=1,use_multiprocessing=True, max_queue_size=20)

#### train for loop

In [14]:
save_dir = 'save_model_BC'
model_name = 'model_10file_timestep_16_'
starting_num = 31
highest_num = 40
for i in range(4, 181):   

    ## LOAD MODEL AND SET LEARNING RATE

    # pick up training from earlier spot
    print(f'-- loading model from saved file model_timestep_16_{i-1} --')
    model = tp_load_model(save_dir, model_name + str(i-1))
    # TODO: if .p file exists, load and check if matches current config
    # hypers_load = pickle.load(open(save_dir+'/'+weights_name+'.p', 'rb'))
    # print(hypers_load)
    print('setting optimiser...')
    with strategy.scope():
        model.optimizer.learning_rate.assign(0.0)
        #K.set_value(model.optimizer.learning_rate, 0.0)
    hist = model.fit(tmp_generator,epochs=1,verbose=1) 
    # have to load the optimiser after we've compiled the model
    # model._make_train_function()
    model_path = os.path.join(save_dir, model_name+ str(i-1)+'_opt.pkl')
    with open(model_path, 'rb') as f:
        opt_weight_values = pickle.load(f)
        with strategy.scope():
            model.optimizer.set_weights(opt_weight_values)
        # opt.set_weights(weight_values)

    # manually set the learning rate in case changed since save
    model.optimizer.learning_rate.assign(l_rate)
    #K.set_value(model.optimizer.learning_rate, l_rate)
    print('updated optimiser to saved state')
    print('starting to train...')

    ## GET NEW DATASET
    print(f'using file {starting_num} --> {highest_num} to train')
    data_list = [str(x1)+'-'+str(x2) for x1 in np.arange(starting_num,highest_num+1) for x2 in np.arange(0,1000-N_TIMESTEPS-int(N_JITTER),N_TIMESTEPS)]
    np.random.shuffle(data_list1) # shuffle it (in place)
    partition = {}
    partition['train'] = data_list1[:int(len(data_list)*1.)]
    partition['validation'] = data_list1[int(len(data_list)*0.995):]
    training_generator = DataGenerator(list_IDs=partition['train'], batch_size=batch_size, shuffle=True)
    validation_generator = DataGenerator(list_IDs=partition['validation'], batch_size=batch_size, shuffle=True)

    ## TRAIN MODEL

    hist = model.fit(training_generator,validation_data=validation_generator,epochs=4,verbose=1) 

    ## SAVE MODEL
    tp_save_model(model, save_dir, model_name + str(i))
    print(f'saved model {model_name + str(i)}')

    # CHANGE STARTING_NUM AND HIGHEST_NUM BY 10 (NEXT 10 FILE)
    starting_num += 10
    highest_num += 10

-- loading model from saved file model_timestep_16_3 --
Loaded model architecture from save_model_BC/model_10file_timestep_16_3.json
Loaded weights from save_model_BC/model_10file_timestep_16_3.weights.h5
setting optimiser...


/home/tuan/.pyenv/versions/3.12.0/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


NameError: name 'x' is not defined

In [3]:
model = tp_load_model(save_dir='save_model_BC', model_name='model_10file_timestep_96_finetune_28_11_19')
model.summary()

I0000 00:00:1732995357.691669     685 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Loaded model architecture from save_model_BC/model_10file_timestep_96_finetune_28_11_19.json
Loaded weights from save_model_BC/model_10file_timestep_96_finetune_28_11_19.weights.h5


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ main_in             │ (None, 96, 150,   │          0 │ -                 │
│ (InputLayer)        │ 412, 3)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 96, 10,    │    867,975 │ main_in[0][0]     │
│ (TimeDistributed)   │ 26, 112)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_lstm2d         │ (None, 96, 8, 24, │  3,392,512 │ time_distributed… │
│ (ConvLSTM2D)        │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_1  │ (None, 96, 49152) │          0 │ conv_lstm2d[0][0] │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_2  │ (None, 96, 11)    │    540,683 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_3  │ (None, 96, 2)     │     98,306 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_4  │ (None, 96, 23)    │  1,130,519 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_5  │ (None, 96, 15)    │    737,295 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_6  │ (None, 96, 1)     │     49,153 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 96, 52)    │          0 │ time_distributed… │
│ (Concatenate)       │                   │            │ time_distributed… │
│                     │                   │            │ time_distributed… │
│                     │                   │            │ time_distributed… │
│                     │                   │            │ time_distributed… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 20,416,997 (77.88 MB)

 Trainable params: 6,800,276 (25.94 MB)

 Non-trainable params: 16,167 (63.16 KB)

 Optimizer params: 13,600,554 (51.88 MB)

In [4]:
model = tp_load_model(save_dir='stateful_model', model_name='model_10file_timestep_96_finetune_28_11_19_stateful')
model.summary()

Loaded model architecture from stateful_model/model_10file_timestep_96_finetune_28_11_19_stateful.json
Loaded weights from stateful_model/model_10file_timestep_96_finetune_28_11_19_stateful.weights.h5


/home/tuan/.pyenv/versions/3.12.0/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 314 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ main_in             │ (1, 1, 150, 412,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (1, 1, 10, 26,    │    867,975 │ main_in[0][0]     │
│ (TimeDistributed)   │ 112)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_lstm2d         │ (1, 1, 8, 24,     │  3,392,512 │ time_distributed… │
│ (ConvLSTM2D)        │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_1  │ (1, 1, 49152)     │          0 │ conv_lstm2d[0][0] │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (1, 1, 11)        │    540,683 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (1, 1, 2)         │     98,306 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (1, 1, 23)        │  1,130,519 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (1, 1, 15)        │    737,295 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (1, 1, 1)         │     49,153 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (1, 1, 52)        │          0 │ dense[0][0],      │
│ (Concatenate)       │                   │            │ dense_1[0][0],    │
│                     │                   │            │ dense_2[0][0],    │
│                     │                   │            │ dense_3[0][0],    │
│                     │                   │            │ dense_4[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 20,416,997 (77.88 MB)

 Trainable params: 6,800,276 (25.94 MB)

 Non-trainable params: 16,167 (63.16 KB)

 Optimizer params: 13,600,554 (51.88 MB)

In [8]:
model = tp_load_model(save_dir='save_model_BC', model_name='model_10file_timestep_16_1')
model.summary()

Loaded model architecture from save_model_BC/model_10file_timestep_16_1.json
Loaded weights from save_model_BC/model_10file_timestep_16_1.weights.h5


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ main_in             │ (None, 32, 150,   │          0 │ -                 │
│ (InputLayer)        │ 412, 3)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 32, 10,    │    867,975 │ main_in[0][0]     │
│ (TimeDistributed)   │ 26, 112)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_lstm2d         │ (None, 32, 8, 24, │  3,392,512 │ time_distributed… │
│ (ConvLSTM2D)        │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_1  │ (None, 32, 49152) │          0 │ conv_lstm2d[0][0] │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_2  │ (None, 32, 11)    │    540,683 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_3  │ (None, 32, 2)     │     98,306 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_4  │ (None, 32, 23)    │  1,130,519 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_5  │ (None, 32, 15)    │    737,295 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_6  │ (None, 32, 1)     │     49,153 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 32, 52)    │          0 │ time_distributed… │
│ (Concatenate)       │                   │            │ time_distributed… │
│                     │                   │            │ time_distributed… │
│                     │                   │            │ time_distributed… │
│                     │                   │            │ time_distributed… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 20,416,997 (77.88 MB)

 Trainable params: 6,800,276 (25.94 MB)

 Non-trainable params: 16,167 (63.16 KB)

 Optimizer params: 13,600,554 (51.88 MB)